In [1]:
import torch  # For PyTorch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3070 Laptop GPU


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Sep__8_19:56:38_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.3, V12.3.52
Build cuda_12.3.r12.3/compiler.33281558_0


In [1]:
import pandas as pd
import pprint
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline

C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pp = pprint.PrettyPrinter(indent=4)
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


In [2]:


# Load the tokenizer and model
model_path = 'nvidia/Minitron-4B-Base'
tokenizer  = AutoTokenizer.from_pretrained(model_path)

device = 'cuda'
dtype  = torch.bfloat16
model  = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=dtype, device_map=device)


In [2]:


model_path = 'nvidia/Minitron-4B-Base'
tokenizer = AutoTokenizer.from_pretrained(model_path)
text_generator = pipeline(
    "text-generation",
    model=model_path,
    tokenizer=tokenizer,  # Pass the tokenizer object
    device=0,
    torch_dtype=torch.bfloat16,
    max_length=10 # Reduce sequence length
)


Device set to use cuda:0


In [12]:
df = pd.read_csv("C:/Users/ghass/Downloads/1000_enron_spam_data.csv")
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
995,995,29351,can you help ! ! !,from : mrs . victoria ndlovu\nattn :\nyou may ...,spam,2004-08-13
996,996,32821,dont let age deter you from great sex !,20 minutes is all it takes\nmore info here . ....,spam,2005-05-13
997,997,14476,proposed retention bonuses for netco,"louise ,\ni am sorry to bother you at such a c...",ham,2001-12-28
998,998,21733,epson inkjet cartridges from 5 . 99,"save up to 75 % on inkjet , laser copier suppl...",spam,2005-04-18


In [13]:
balanced_df = df[:110]

In [14]:
spam_count = (balanced_df['Spam/Ham'] == 'spam').sum()
ham_count = (balanced_df['Spam/Ham'] == 'ham').sum()

print(f"Spam count: {spam_count}")
print(f"Ham count: {ham_count}")

Spam count: 52
Ham count: 58


In [15]:
df = balanced_df
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
105,105,24490,have skin like a model,we ' ll\ngive you your first tube of body scul...,spam,2002-07-22
106,106,3559,fw : sitara eol bridge problem today,fyi > > > we were also monitoring the eol to s...,ham,2001-10-25
107,107,8555,"ken ,","attached is a correction to pages 10 , 11 , an...",ham,2001-02-20
108,108,29231,winning notification,dalobica lotto bv .\ninternational promotion /...,spam,2004-08-02


In [16]:
import pprint
# Parameters for text generation
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": True,
}

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    text_template = (
        "Classify the text into Spam or Ham. You are going to answer with Class of the email: Spam or Class of the email: Spam"
        f"Text: {row['Message']} Class of the email:?"
    )

    # Generate the output using the pipeline with additional arguments
    generated_text = text_generator(text_template, **generation_args)[0]['generated_text']

    print(f"example {index} : \n")
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Answer: " in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Answer: ", 1)[1].strip().split('\n')[0]  # Take the first line after "Answer: "

    pprint.pprint(email_class)
    print("\n")

    # Map the class to the appropriate format
    if email_class in ["Spam", "Spam."]:
        email_class = "spam"
    elif email_class == "Ham":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)

    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


example 0 : 

(' can use any language you want.\n'
 '\n'
 'Spam\n'
 '\n'
 'Ham\n'
 '\n'
 'The email is from a spammer.\n'
 'Answer: Spam\n'
 '\n'
 'In this task, you are given a set of context paragraphs, some supporting '
 'facts and an answer of a question. Your task is to generate question for '
 'given answer based on set of context paragraphs, supporting facts and an '
 'answer.\n'
 '\n'
 'Context_1 : The 2016–17 season is the 120th season in')


'Spam'


'ham'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


example 1 : 

(' can use the following categories:\n'
 '\n'
 '1. Spambot\n'
 '2. Ham\n'
 '3. Spam\n'
 '4. Ham\n'
 '\n'
 'The text is about a person who is trying to sell their car.\n'
 'Answer: 3\n'
 '\n'
 'The text is about a person who is trying to sell their car.\n'
 '\n'
 'What are the benefits of using a VPN?\n'
 'Answer: A VPN (Virtual Private Network) is a technology that allows you to '
 'connect to the internet securely and anonymously. There are')


'3'


'spam'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

example 106 : 

(' are expected to report only one label per row. The first column is the '
 'label, the second column is the text.\n'
 '\n'
 'Label: Spam\n'
 "Text: I'm not sure what to do with this. I've been trying to get it to work "
 "for a while, but I can't seem to get it to work. I've tried everything I can "
 "think of, but it just doesn't work. I'm not sure what to do.\n"
 'Answer: Spam')


'Spam'


'ham'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


example 107 : 

(' can use the following words: "Spam", "Ham", "Spam mail", "Ham mail".\n'
 '\n'
 '[SPAM]\n'
 '\n'
 '[Spam]\n'
 'Answer: Spam\n'
 '\n'
 'In this task, you are given a sentence with a missing word that can be an '
 'object, a person, and/or an action. Fill in the blank with a plausible word. '
 'Although each sentence has many correct answers, you only have to write one '
 'answer.\n'
 '\n'
 'PersonX takes ___ to')


'Spam'


'ham'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


example 108 : 

(' are expected to report only if the sentence has the word "spam".\n'
 '\n'
 "I'm not sure if I'm supposed to be offended or not.\n"
 'Answer: Ham\n'
 '\n'
 'In this task, you are given two phrases: Head and Tail, separated with '
 '<sep>. The Head and the Tail events are short phrases possibly involving '
 'participants. The names of specific people have been replaced by generic '
 'words (e.g., PersonX, PersonY, PersonZ). PersonX is always the')


'Ham'


'spam'
example 109 : 

(' may use the following text to help you make your decision:\n'
 '\n'
 'Text:  The National Institute for the Deaf (NID) is a research institute in '
 'the Netherlands. It is located in the city of Utrecht.\n'
 'Answer: Ham\n'
 '\n'
 'In this task, you are given a sentence with a missing word that can be an '
 'object, a person, and/or an action. Fill in the blank with a plausible word. '
 'Although each sentence has many correct answers, you only have to write')


'Ham'


'ham'


In [17]:
#26 minutes
unique_list = list(set(predicted_labels))
unique_list

['ham',
 '0',
 '3',
 "I'm looking for a job as a software engineer. I have 5 years of experience in this field and I'm very interested in working",
 'Not spammer',
 'Not a spammer',
 'You can use the following categories:',
 '1',
 'None',
 'Other',
 'spam',
 'Spambot',
 'Not Spam']

In [18]:
predicted_labels_1 = ["spam" if label == 'SPAM' or label == 'spam.' or label =='Spambot'  else label for label in predicted_labels]
predicted_labels_1 = ["ham" if label == 'HAM' or label == 'Not a spammer' or label == 'Not spammer' or label=='Not Spam' else label for label in predicted_labels]

In [20]:
#26 minutes
unique_list = list(set(predicted_labels_1))
unique_list

['ham',
 '0',
 '3',
 "I'm looking for a job as a software engineer. I have 5 years of experience in this field and I'm very interested in working",
 'You can use the following categories:',
 '1',
 'None',
 'Other',
 'spam',
 'Spambot']

In [22]:
label_mapping = {'ham': 0, 'spam': 1}
# Define valid labels
valid_labels = set(label_mapping.keys())  # e.g., {'spam', 'ham'}

# Filter true_labels and predicted_labels to remove invalid entries
filtered_pairs = [
    (true, pred)
    for true, pred in zip(true_labels, predicted_labels_1)
    if true in valid_labels and pred in valid_labels
]

# Unpack the filtered pairs
true_labels_filtered, predicted_labels_filtered = zip(*filtered_pairs) if filtered_pairs else ([], [])

# Map filtered labels using the label_mapping dictionary
true_labels_mapped = [label_mapping[label] for label in true_labels_filtered]
predicted_labels_mapped = [label_mapping[label] for label in predicted_labels_filtered]

In [23]:
len(predicted_labels_mapped)

81

In [24]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Calculate F1 Score
f1 = f1_score(true_labels_mapped, predicted_labels_mapped, average='binary')  # Use 'binary' for binary classification
print("F1 Score:", f1)

# Calculate Accuracy
accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
print("Accuracy:", accuracy)

# Calculate Precision
precision = precision_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Precision:", precision)

# Calculate Recall
recall = recall_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Recall:", recall) 

F1 Score: 0.4810126582278481
Accuracy: 0.49382716049382713
Precision: 0.48717948717948717
Recall: 0.475


In [37]:
import pprint

# Parameters for text generation
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": True,
}

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message (prompt)
    # Assign the prompt to a variable
    prompt = """System: I want you to act as an email classifier as Spam or Ham.\n\n User: {}\n\n Class of the email is   """.format(row["Message"])


    
    # Generate the output using the pipeline with additional arguments
    generated_text = text_generator(prompt, **generation_args)[0]['generated_text']
    
    # Print the generated response for debugging
    print(f"Example {index}:\n")
    pprint.pprint(generated_text)
    print("\n")
    
    # Extract the class of the email
    email_class = None
    if "class of email:" in generated_text:
        try:
            # Extract the portion of the text after "class of email:"
            email_class = generated_text.split("class of email:", 1)[1].strip().split('\n')[0].strip(".")
        except IndexError:
            email_class = "other"  # Default if parsing fails
    
    pprint.pprint(email_class)  # Debugging print
    print("\n")
    
    # Map the extracted class to the desired format
    if email_class in ["Spam", "Spam."]:
        email_class = "spam"
    elif email_class in ["Ham", "Ham."]:
        email_class = "ham"
    else:
        email_class = "other"
    
    # Compare with the actual label from the DataFrame
    actual_class = row["Spam/Ham"]  # Assuming the actual label column is named "Spam/Ham"
    pprint.pprint(actual_class)  # Debugging print
    
    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)

# At the end, you could calculate metrics if needed
# For example: accuracy = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p]) / len(true_labels)


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Example 0:

(' system. I want you to send me an email every time I receive an email.\n'
 '\n'
 "System: I'm sorry, but I cannot do that.\n"
 '\n'
 'User: Why not?\n'
 '\n'
 'System: Because I am a system. I cannot act as an email system.\n'
 '\n'
 'User: Can you act as a system that acts as an email system?\n'
 '\n'
 'System: I am sorry, but I cannot do that either.\n'
 '\n'
 'User: Why not?\n'
 '\n'
 'System:')


None


'ham'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Example 1:

(' system.\n'
 '\n'
 'I want you to send me an email every time I ask you to.\n'
 '\n'
 'I want you to send me an email every time I ask you to.\n'
 '\n'
 'I want you to send me an email every time I ask you to.\n'
 '\n'
 'I want you to send me an email every time I ask you to.\n'
 '\n'
 'I want you to send me an email every time I ask you to.\n'
 '\n'
 'I want you to send me an email every time I')


None


'spam'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Example 106:

(' client for a while.\n'
 '\n'
 "You: I'm sorry, but I don't understand.\n"
 '\n'
 'System: I want you to act as an email client for a while.\n'
 '\n'
 "You: I'm sorry, but I don't understand.\n"
 '\n'
 'System: I want you to act as an email client for a while.\n'
 '\n'
 "You: I'm sorry, but I don't understand.\n"
 '\n'
 'System: I want you to act as an email client')


None


'ham'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Example 107:

(' system for the company. You will receive emails from the company and send '
 'them to the appropriate person.\n'
 '\n'
 'Write a letter to your friend, who is going to visit you.\n'
 '\n'
 'Letter: Dear [Friend],\n'
 'I am so excited that you are coming to visit me! I can’t wait to show you '
 'around my city and introduce you to my friends. I hope you will have a great '
 'time here, and I look forward to seeing you soon.\n'
 'Answer: Hi [Friend')


None


'ham'


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Example 108:

(' server for my system.\n'
 '\n'
 "User: I'm not sure I understand. Can you explain it in more detail?\n"
 '\n'
 'System: I want you to act as an email server for my system.\n'
 '\n'
 "User: I'm not sure I understand. Can you explain it in more detail?\n"
 '\n'
 'System: I want you to act as an email server for my system.\n'
 '\n'
 "User: I'm not sure I understand. Can you explain it in more detail?\n")


None


'spam'
Example 109:

(' filter.\n'
 '\n'
 'User: What is the purpose of an email filter?\n'
 'Answer: The purpose of an email filter is to automatically sort incoming '
 'emails into different categories based on their content. This can be useful '
 'for managing large amounts of email, or for prioritizing emails based on '
 'their importance.')


None


'ham'


In [40]:
label_mapping = {'ham': 0, 'spam': 1}
# Define valid labels
valid_labels = set(label_mapping.keys())  # e.g., {'spam', 'ham'}

# Filter true_labels and predicted_labels to remove invalid entries
filtered_pairs = [
    (true, pred)
    for true, pred in zip(true_labels, predicted_labels)
    if true in valid_labels and pred in valid_labels
]

In [41]:
# Unpack the filtered pairs
true_labels_filtered, predicted_labels_filtered = zip(*filtered_pairs) if filtered_pairs else ([], [])

# Map filtered labels using the label_mapping dictionary
true_labels_mapped = [label_mapping[label] for label in true_labels_filtered]
predicted_labels_mapped = [label_mapping[label] for label in predicted_labels_filtered]
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Calculate F1 Score
f1 = f1_score(true_labels_mapped, predicted_labels_mapped, average='binary')  # Use 'binary' for binary classification
print("F1 Score:", f1)

# Calculate Accuracy
accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
print("Accuracy:", accuracy)

# Calculate Precision
precision = precision_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Precision:", precision)

# Calculate Recall
recall = recall_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Recall:", recall) 

F1 Score: 0.0
Accuracy: nan
Precision: 0.0
Recall: 0.0


C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\numpy\lib\_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ghass\

### Comparaison des prompts pour Minitron_4B_Base(Type et Performance) ###

On n'a pas utilisé de role car le modèle Minitron ne supporte pas les roles comme user and system.
# 1ère formulation :
Explication de la tache a accomplir de classification en premier. puis on donne le type de réponse à en donner. 
     
        text_template = (
        "Classify the text into Spam or Ham. You are going to answer with Class of the email: Spam or Class of the email: Spam"
        f"Text: {row['Message']} Class of the email:?"
    )
# 2ème formulation :

    """System: I want you to act as an email classifier as Spam or Ham.\n\n User: {}\n\n Class of the email is   """.format(row["Message"])
 
 ici la différence est qu'on a essayé de simuler les roles user et System dans notre prompt. De plus on a donné la forme de la réponse.
# Tableau comparatif des performances
| Formulation | Accuracy | F1 Score | Precision | Recall |
|-------------|----------|----------|-----------|--------|
| **1**       | 48.1%   | 49.38%   | 48.71%    | 47.5% |
| **2**       | 0.00%   | 0.00%   | 0.00%    | 0.00%  |


  # => 
  La 1ère formulation est faible en terme des performances puique le F1 score et accuracy sont autours de 50%
  Cependant la 2ème formulation a échoué de classifier l'email . On peut conclure qu'on ne peut pas simuler les roles dans nos prompts. Ce modèle en total n'est pas spécialisé dans cette tache de classification.


## Analyse des erreurs
Example 55 il trouve des termes qu’il n’est pas abitué a trouver dans le mail alors il estime que ce sont des spams. 
Example 66 il trouve une repetition du meme terme dans l’email donc il a cru que c’est un spam

- Il y a des halucinations du model aussi puisque il commence sa réponse par  l’utilisation de langue (Englais ou Hindi) example 85 et englais et espagnol example 78 qui n’est pa relié a la teche alors que le texte en englais 
- Exemple 82 : il trouve que les promotions et liens qui existent dans l’email sont considérés comme spam alors que cela n’est pas dans tous les cas corrects puisque on peut avoir des liens dans les emails valides.
# => 	
 Le modèle ne peut pas performer bien sur ces exemples d’email il manque de connaissance sur la forme d’email et le type de contenu comme les liens.
